PROGRAMA_ANALISIS_LactAPP

Version: 05/12/2023

Test de funcionamiento Neo4j para analisis.

Necesario:

    - Docker Neo4j arrancado

    - Activado entorno venv

    - Licencia OpenAI (Archivo: .env)

Version 1.01

In [2]:
# Bibliotecas a Cargar

# Group 1: Web Framework (Flask) / Grupo 1: Framework Web (Flask)
from flask import Flask, request, jsonify, render_template
# Flask is a Python web framework used for building web applications.
# Imports include essential components for creating routes, handling HTTP requests, and rendering HTML templates.
# Flask es un framework web de Python que se utiliza para crear aplicaciones web.
# Las importaciones incluyen componentes esenciales para crear rutas, manejar solicitudes HTTP y renderizar plantillas HTML.

# Group 2: System Operations and Environment Variables / Grupo 2: Operaciones del Sistema y Variables de Entorno
import os
from dotenv import load_dotenv, find_dotenv
# This group handles system related operations and environment variables.
# 'os' provides functions for interacting with the operating system.
# 'dotenv' allows loading environment variables from '.env' files.
# Este grupo maneja operaciones relacionadas con el sistema y variables de entorno.
# 'os' proporciona funciones para interactuar con el sistema operativo.
# 'dotenv' permite cargar variables de entorno desde archivos '.env'.

# Group 3: Natural Language Processing (NLP) / Grupo 3: Procesamiento de Lenguaje Natural (NLP)
from langchain import (
    PromptTemplate, LLMChain, llms, text_splitter, vectorstores, chains, embeddings
)
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
# This group focuses on natural language processing (NLP) and utilizes the 'langchain' library.
# It includes various functionalities for text processing and language models.
# 'SentenceTransformer' is used for generating sentence vector representations for similarity calculations.
# 'cosine_similarity' is used to calculate cosine similarity between vectors.
# Este grupo se centra en el procesamiento de lenguaje natural (NLP) y utiliza la biblioteca 'langchain'.
# Incluye diversas funcionalidades para el procesamiento de texto y modelos de lenguaje.
# 'SentenceTransformer' permite generar representaciones vectoriales de oraciones para cálculos de similitud.
# 'cosine_similarity' se usa para calcular la similitud de coseno entre vectores.

# Group 4: Data Loading and Database Operations / Grupo 4: Carga de Datos y Operaciones de Base de Datos
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.graphs import Neo4jGraph
from neo4j import GraphDatabase
# This group is responsible for loading documents and handling database operations.
# It includes functionalities for loading PDF documents ('PyPDFLoader') and working with databases like Neo4j.
# Este grupo se encarga de cargar documentos y manejar operaciones de base de datos.
# Incluye funcionalidades para cargar documentos PDF ('PyPDFLoader') y trabajar con bases de datos como Neo4j.

# Group 5: Additional Language Processing / Grupo 5: Procesamiento de Lenguaje Adicional
from langchain.llms import CTransformers
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.prompt_selector import ConditionalPromptSelector
# This group includes additional imports related to 'langchain' and language models,
# like 'ChatOpenAI' for generating chat responses.
# Este grupo incluye importaciones adicionales relacionadas con 'langchain' y modelos de lenguaje,
# como 'ChatOpenAI' para generar respuestas de chat.


/media/ubuntu/Linux3/CHAT_BOT_FINANCIERO/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## Caso de usar .env file
## In case of using a .env file

# Carga de variables de Entorno 
# Load environment variables

load_dotenv(find_dotenv(), override=True)

# Accede a las variables de entorno en tu código
# Access environment variables in your code
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
URL_NEO4J = os.getenv('URL_NEO4J')
USERNAME_NEO4J = os.getenv('USERNAME_NEO4J')
PASSWORD_NEO4J = os.getenv('PASSWORD_NEO4J')

# Ahora, puedes usar las variables en tu script
# Now, you can use the variables in your script
#print(f'OPENAI_API_KEY: {OPENAI_API_KEY}')
#print(f'URL_NEO4J: {URL_NEO4J}')
#print(f'USERNAME_NEO4J: {USERNAME_NEO4J}')
#print(f'PASSWORD_NEO4J: {PASSWORD_NEO4J}')

1.- CARGA MODELO LOCAL

Modelo Local -> llm

In [4]:
# Initialize LLM and other components as in the original code
# Inicializa el LLM y otros componentes como en el código original

# Uncomment and set the appropriate model name for your local environment
# Descomenta y establece el nombre del modelo adecuado para tu entorno local

# local_llm = "neural-chat-7b-v3-1.Q4_K_M.gguf"
local_llm = "openhermes-2.5-mistral-7b.Q3_K_M.gguf"

In [5]:
# Model Parameters
# Parámetros del Modelo

# Define configuration parameters for the model
# Define los parámetros de configuración para el modelo
config = {
    'max_new_tokens': 512,       # Maximum number of new tokens generated
    'repetition_penalty': 1.1,   # Repetition penalty for text generation
    'temperature': 0,           # Temperature for controlling randomness (0 for deterministic output)
    'context_length': 1024,     # Length of the input context
    'stream': False             # Whether to stream the output or generate it all at once
}

| Parameter            | Type      | Description                                                     | Default   |
|----------------------|-----------|-----------------------------------------------------------------|-----------|
| top_k                | int       | The top-k value to use for sampling.                            | 40        |
| top_p                | float     | The top-p value to use for sampling.                            | 0.95      |
| temperature          | float     | The temperature to use for sampling.                            | 0.8       |
| repetition_penalty   | float     | The repetition penalty to use for sampling.                     | 1.1       |
| last_n_tokens        | int       | The number of last tokens to use for repetition penalty.        | 64        |
| seed                 | int       | The seed value to use for sampling tokens.                      | -1        |
| max_new_tokens       | int       | The maximum number of new tokens to generate.                   | 256       |
| stop                 | List[str] | A list of sequences to stop generation when encountered.        | None      |
| stream               | bool      | Whether to stream the generated text.                           | False     |
| reset                | bool      | Whether to reset the model state before generating text.        | True      |
| batch_size           | int       | The batch size to use for evaluating tokens in a single prompt. | 8         |
| threads              | int       | The number of threads to use for evaluating tokens.             | -1        |
| context_length       | int       | The maximum context length to use.                              | -1        |
| gpu_layers           | int       | The number of layers to run on GPU.                             | 0         |

In [6]:
# Initialize a CTransformers object with the specified parameters.
# Inicializa un objeto CTransformers con los parámetros especificados.
llm = CTransformers(
    model=local_llm,
    model_type="mistral",
    lib="avx2",
    **config
)

# Print a message to indicate that LLM (Language Model) has been initialized.
# Imprime un mensaje para indicar que LLM (Modelo de Lenguaje) ha sido inicializado.
print("LLM Initialized....")

LLM Initialized....


2- MODELO EMBEDDINGS

In [7]:
# Define the model name for embeddings.
# Define el nombre del modelo para los embeddings.
model_name = "BAAI/bge-large-en"

# Specify keyword arguments for the model (e.g., device).
# Especifica argumentos clave para el modelo (por ejemplo, dispositivo).
model_kwargs = {'device': 'cpu'}

# Specify keyword arguments for encoding (e.g., normalization settings).
# Especifica argumentos clave para la codificación (por ejemplo, configuración de normalización).
encode_kwargs = {'normalize_embeddings': False}

# Initialize HuggingFaceBgeEmbeddings with the specified parameters.
# Inicializa HuggingFaceBgeEmbeddings con los parámetros especificados.
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

3.- DEFINICION PROMPTS

Para modelo local

Nota: El modelo local se utilizara en caso de que queramos generar una respuesta en modo texto usando el contexto de la cadana identificada como mas relevante para la pregunta indicada por la usuaria.

In [8]:
# Define a prompt template for generating prompts.
# Define una plantilla de prompt para generar preguntas.
prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

# Specify the input variables used in the template.
# Especifica las variables de entrada utilizadas en la plantilla.
input_variables = ['context', 'question']

# Create a PromptTemplate object with the specified template and input variables.
# Crea un objeto PromptTemplate con la plantilla y las variables de entrada especificadas.
prompt = PromptTemplate(template=prompt_template, input_variables=input_variables)

4.- DATOS CONEXION Neo4j

La estructura de árboles de la aplicación LactApp se encuentra en una BBDD Neo4j

Hemos procedido a:

- Generar BBDD Neo4j en Docker

- Cargar sobre esa BBDD el DUMP proporcionado con los datos de la aplicación.

In [9]:
# Establish a connection to the Neo4j database.
# Establece una conexión a la base de datos Neo4j.
url = URL_NEO4J
username = USERNAME_NEO4J
password = PASSWORD_NEO4J

In [10]:
# Function to check the connection to the Neo4j database
# Función para verificar la conexión a la base de datos Neo4j
def verificar_conexion(url, username, password):
    try:
        # Attempt to establish a connection to the Neo4j database
        # Intento de establecer una conexión a la base de datos Neo4j
        with GraphDatabase.driver(url, auth=(username, password)) as driver:
            with driver.session() as session:
                # Run a test query to check the connection
                # Ejecutar una consulta de prueba para verificar la conexión
                session.run("RETURN 1")
        return True  # Return True if the connection is successful
    except Exception as e:
        # Handle any exceptions that occur during the connection attempt
        # Manejar cualquier excepción que ocurra durante el intento de conexión
        print(f"Connection error: {str(e)}")
        return False  # Return False if the connection fails

# Check the connection to the Neo4j database
# Verifica la conexión a la base de datos Neo4j
if verificar_conexion(url, username, password):
    print("Successful connection to the Neo4j database")
else:
    print("Failed to connect to the Neo4j database")

Successful connection to the Neo4j database


5.- INICIALIZACION EMBEDDING DE LA BBDD Neo4j

Procedemos a realizar el embedding de todos los nodos (Solo la primera vez)

Utilizaremos el modelo de embedding inicializado --> embeddings

Luego solo se realizara el embedding de los nodos que no dispongan de embedding ya calculado.

In [11]:
# Function to initialize the embedding for a node type
# Función para inicializar el embedding para un tipo de nodo
def init_node_embedding(node_label, text_properties):
    return Neo4jVector.from_existing_graph(
        embedding=embeddings,  # The embedding to use for initialization
        url=url,  # URL for the Neo4j database
        username=username,  # Username for database authentication
        password=password,  # Password for database authentication
        index_name=f"{node_label.lower()}_index",  # Name of the index for the node label
        node_label=node_label,  # Label of the node type
        text_node_properties=text_properties,  # Text properties for node embedding
        embedding_node_property="NEWembedding"  # Property name for the embedding in the database
    )

In [12]:
# Initialize embeddings for different types of nodes
# Inicializar embeddings para diferentes tipos de nodos

# Initialize embedding for "Choice" nodes with support for Spanish strings
Choice_embedding = init_node_embedding("Choice", ["string_es"])
print("Choice embedding OK")  # Print message to indicate successful initialization
# Inicializar la incrustación para nodos "Choice" con soporte para cadenas en español
# Imprimir mensaje para indicar una inicialización exitosa

# Initialize embedding for "Question" nodes with support for Spanish strings
Question_embedding = init_node_embedding("Question", ["string_es"])
print("Question embedding OK")  # Print message to indicate successful initialization
# Inicializar la incrustación para nodos "Question" con soporte para cadenas en español
# Imprimir mensaje para indicar una inicialización exitosa

# Initialize embedding for "Theme" nodes with support for Spanish strings
Theme_embedding = init_node_embedding("Theme", ["string_es"])
print("Theme embedding OK")  # Print message to indicate successful initialization
# Inicializar la incrustación para nodos "Theme" con soporte para cadenas en español
# Imprimir mensaje para indicar una inicialización exitosa

# Initialize embedding for "Reply" nodes with support for Spanish strings
Reply_embedding = init_node_embedding("Reply", ["string_es"])
print("Reply embedding OK")  # Print message to indicate successful initialization
# Inicializar la incrustación para nodos "Reply" con soporte para cadenas en español
# Imprimir mensaje para indicar una inicialización exitosa

Choice embedding OK
Question embedding OK
Theme embedding OK
Reply embedding OK


6.- CALCULO SIMILITUD ENTRE EMBEDDING BBDD Neo4j y QUERY

Generaremos una propiedad 'similarity_query' en cada nodo.

El valor de esa propiedad sera la similitud entre el vector de la query y el EMBEDDING almacenado en cada nodo.

Se actualizara cada vez que se plantee una query nueva.

In [30]:
# Function to calculate similarity between the query and nodes
# Función para calcular la similitud entre la consulta y los nodos
def calculate_similarity(tx, input_query):
    # Get embeddings for the query
    # Obtener representaciones incrustadas (embeddings) para la consulta
    query_embedding = embeddings.encode(input_query, **encode_kwargs)

    # Obtain information for all nodes
    # Obtener información de todos los nodos
    result = tx.run("""
    MATCH (n)
    RETURN ID(n) AS node_id, n.NEWembedding AS node_embedding
    """)

    queries_to_update = []

    for record in result:
        node_id = record["node_id"]
        node_embedding = record["node_embedding"]

        # Calculate cosine similarity between the query and the node
        # Calcular la similitud del coseno entre la consulta y el nodo
        similarity_score = cosine_similarity([query_embedding], [node_embedding])[0][0]

        # Prepare the query to update the node
        # Preparar la consulta para actualizar el nodo
        queries_to_update.append({
            "node_id": node_id,
            "score": similarity_score
        })

    # Update all nodes in a single transaction
    # Actualizar todos los nodos en una sola transacción
    for query_data in queries_to_update:
        tx.run("""
        MATCH (n)
        WHERE ID(n) = $node_id
        SET n.similarity_query = $score
        """, node_id=query_data["node_id"], score=query_data["score"])

7.- CALCULO NODOS MAS CERCANOS SEGUN MEDIA SIMILARIDAD INDICADA

- calculate_top_similarity_nodes --> Identifica los 5 nodos mas cercanos y lo devuelve en forma de lista 

In [14]:
# ESTA MODIFICADO EN LA VERSION LIMPIA
# Function to calculate the list of closest nodes
# Función para calcular la lista de nodos más cercanos
def calculate_top_similarity_nodes(tx, input_query):
    # Get embeddings for the query
    query_embedding = embeddings.encode(input_query, **encode_kwargs)

    result = tx.run("""
    MATCH (n)
    RETURN n.node_id AS node_id, n.NEWembedding AS node_embedding, n.similaridadquery AS similarity_score, n.string_es AS node_text
    ORDER BY similarity_score DESC
    LIMIT 5
    """)

    top_nodes = []

    for record in result:
        node_id = record["node_id"]
        node_embedding = record["node_embedding"]
        similarity_score = record["similarity_score"]
        textoNodo = record["node_text"]

        top_nodes.append({
            "node_id": node_id,
            "node_embedding": node_embedding,
            "similarity_score": similarity_score,
            "texto": textoNodo
        })

    return top_nodes

8.- CLASE PARA CALCULO CAMINOS OPTIMOS

Clase para identificar caminos optimos
    
    find_path -> Dado Nodo de inicio, profundidad maxima de busqueda devuelve lista de información de nodos para los caminos encontrados.
    
    get_all_themes_string -->  lista de cadenas de todos los Themas de la Neo4j
    
     get_node_ids_by_theme_string --> Devuelve una lista de IDs de nodos asociados con la cadena de tema proporcionada

In [15]:
class GraphPathFinder:
    def __init__(self, url, user, password):
        # Initialize the GraphPathFinder class with database connection parameters
        # Inicializar la clase GraphPathFinder con parámetros de conexión a la base de datos
        self.driver = GraphDatabase.driver(url, auth=(user, password))

    def close(self):
        # Close the database connection
        # Cerrar la conexión a la base de datos
        self.driver.close()

    def find_paths(self, start_node_id, max_depth, external_embedding):
        with self.driver.session() as session:
            query = """
            MATCH path = (n {node_id: $start_node_id})-[*1..%s]-(end)
            WHERE all(node in nodes(path) WHERE node.similaridadquery >= 0.7)
            RETURN [node in nodes(path) | [node.node_id, node.similaridadquery, node.string_es]] AS node_info
            ORDER BY reduce(s = 0, n IN nodes(path) | s + n.similaridadquery) DESC
            """ % max_depth
            result = session.run(query, start_node_id=start_node_id)
            # Return a list of node information for the found paths
            # Devolver una lista de información de nodos para los caminos encontrados
            return [record["node_info"] for record in result]

    def get_all_theme_strings(self):
        with self.driver.session() as session:
            query = """
            MATCH (theme:Theme)
            RETURN collect(theme.string_es) AS theme_strings
            """
            result = session.run(query)
            # Return a list of all theme strings in the database
            # Devolver una lista de todas las cadenas de temas en la base de datos
            return [record["theme_strings"] for record in result][0]

    def get_node_ids_by_theme_string(self, theme_string):
        with self.driver.session() as session:
            query = """
            MATCH (node:Theme {string_es: $theme_string})
            RETURN collect(node.node_id) AS node_ids
            """
            result = session.run(query, theme_string=theme_string)
            # Return a list of node IDs associated with the given theme string
            # Devolver una lista de IDs de nodos asociados con la cadena de tema proporcionada
            return [record["node_ids"] for record in result][0]

9.- PROCESADO NODOS

procesar_nodos --> Dado un nodo inicial, una profundidad máxima de nodo, y un número maximo de nodos a extraer nos devuelve la información.

In [16]:
def procesar_nodos1(finder, start_node_ids, Profund_Maxima_Nodos, n_top_paths, query, clasificacion, external_embedding=None):
    """
    finder --> Clase de analisis caminos
    start_nodes_ids --> Lista de Nodos iniciales donde empezar la busqueda
    Profund_Maxima_Nodos --> máximo nivel de profundidad de los nodos
    n_top_paths --> Numero de caminos a devolver
    query --> Consulta del usuario
    external_embedding --> A utilizar en el futuro
    """

    resultados = []  # Almacenará los resultados finales

    # Itera sobre los nodos iniciales
    for start_node_id in start_node_ids:
        # Encuentra los caminos desde el nodo inicial
        paths = finder.find_paths(start_node_id, Profund_Maxima_Nodos, external_embedding)

        # Calcula la media de similaridad y almacena junto con el camino
        path_with_mean = []
        for path in paths:
            if path:
                # Calcula la media de la similaridad en el camino
                mean_similarity = sum(node[1] for node in path if node[1] is not None) / len(path)
                path_with_mean.append((path, mean_similarity))

        # Ordena los caminos por la media de similaridad de mayor a menor
        path_with_mean.sort(key=lambda x: x[1], reverse=True)

        # Guarda la información de los n caminos con mayor media
        top_paths = path_with_mean[:n_top_paths]
        for path, mean in top_paths:
            path_info = [(node[0], node[1]) for node in path]
            string_es_values = [node[2] for node in path if node[2] is not None]
            max_similarity_node = max(path, key=lambda x: x[1])
            max_similarity_node_info = (max_similarity_node[0], max_similarity_node[1], max_similarity_node[2])

            resultados.append({
                'start_node_id': start_node_id,
                'path_info': path_info,
                'mean_similarity': mean,
                'string_es_values': string_es_values,
                'max_similarity_node_info': max_similarity_node_info,
                'query': query,
                'clasificacion': clasificacion
            })

    return resultados  # Devuelve los resultados finales

In [46]:
# Visualización de resultados formato texto.

def mostrar_resultado(resultado, mostrar):
    ''' Imprime el resultado de la búsqueda de caminos en la base de datos Neo4j
    limitando el número de nodos del camino mostrados
    y valores de string_es a mostrar.
    El valor límite dado por: 'mostrar'
    '''
    print("ID del Nodo Inicial:", resultado['start_node_id'])

    # Asegurarse de que num_nodos no exceda la longitud de path_info
    num_nodos = min(len(resultado['path_info']), mostrar)
    print("\nInformación del Camino (mostrando hasta {} nodos):".format(num_nodos))
    for node in resultado['path_info'][:num_nodos]:
        print(f"  Nodo ID: {node[0]}, Similaridad: {node[1]}")

    print("\nMedia de Similaridad:", resultado['mean_similarity'])

    # Asegurarse de que num_valores_string no exceda la longitud de string_es_values
    num_valores_string = min(len(resultado['string_es_values']), mostrar)
    print("\nValores de string_es (mostrando hasta {} valores):".format(num_valores_string))
    for value in resultado['string_es_values'][:num_valores_string]:
        print(f"  {value}")

    print("\nInformación del Nodo con Máxima Similaridad:")
    max_node_info = resultado['max_similarity_node_info']
    print(f"  Nodo ID: {max_node_info[0]}, Similaridad: {max_node_info[1]}")
    print(f"  string_es: {max_node_info[2]}")

A.- PROCESO COMPLETO DE ANALISIS EN SECUENCIA

A1.- Identificación Mensajes Usuarias disponibles

In [18]:
FILEDATOS = '/media/ubuntu/Linux3/02_TFM/TEST_BIBLIOTECA/MODELOS/a.xlsx'

import pandas as pd
import numpy as np
df = pd.read_excel(FILEDATOS)

In [19]:
mensajes = df['message'].to_list()
mensajes = list(set(mensajes))

A2.- GENERACION DE CLASIFICACION MENSAJE USUARIA USANDO LLMs

In [20]:
# Carga Modelo Open AI

#modelOpenAI = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

modelOpenAI = ChatOpenAI(temperature=0)

In [21]:
# Ejemplo de Prompts condicionales

#Ref: https://python.langchain.com/docs/guides/local_llms
# Mistral --> https://smith.langchain.com/hub/rlm/rag-prompt-mistral
# Llama --> https://smith.langchain.com/hub/rlm/rag-prompt-llama


DEFAULT_MISTRAL_SEARCH_PROMPT = PromptTemplate(
    input_variables=["texto", "categorias"],
    template="""<s> [INST] You are an expert in breastfeeding.\
If you don't know the answer, just say that you don't know. \
The output must be only the category. [/INST] </s> \
[INST] Question: classify the text: {texto}. \
Context: The categories for classify are: {categorias}. \
Answer: [/INST]""",
)

DEFAULT_SEARCH_PROMPT = PromptTemplate(
    input_variables=["texto", "categorias"],
    template=""""Clasifica el siguiente texto en una de las siguientes categorías {categorias}: \n Texto: {texto}\nCategoría: """,
)

QUESTION_PROMPT_SELECTOR = ConditionalPromptSelector(
    default_prompt=DEFAULT_SEARCH_PROMPT,
    conditionals=[
        (lambda model: (model == llm), DEFAULT_MISTRAL_SEARCH_PROMPT)
    ],
)


prompt2 = QUESTION_PROMPT_SELECTOR.get_prompt(llm)
prompt3 = QUESTION_PROMPT_SELECTOR.get_prompt(modelOpenAI)



print('Mistral7:\n', prompt2)

print('OpenAI:\n', prompt3)

Mistral7:
 input_variables=['categorias', 'texto'] template="<s> [INST] You are an expert in breastfeeding.If you don't know the answer, just say that you don't know. The output must be only the category. [/INST] </s> [INST] Question: classify the text: {texto}. Context: The categories for classify are: {categorias}. Answer: [/INST]"
OpenAI:
 input_variables=['categorias', 'texto'] template='"Clasifica el siguiente texto en una de las siguientes categorías {categorias}: \n Texto: {texto}\nCategoría: '


In [22]:
# Caso Mistral y Llama2

chainMistral = prompt2 | llm | StrOutputParser()

chainOpenAI = prompt3 | modelOpenAI | StrOutputParser()

In [23]:
def clasificar_LLMs(texto, categorias, chain):
    '''Función de clasificación
    Entrada:
        texto -> Conversación Usuaria
        categorias --> Lista de categorias donde se ha de clasificar
        chain --> Cadena de LangChain
    respuesta:
        Respuesta del modelo
    '''
    response = chain.invoke({"categorias": categorias, "texto": texto})
    return response

In [25]:
# Mensaje de TEST

#entrada = mensajes[0]
entrada = 'Hola! Es normal que mi bebé de 6 semanas quiera comer a cada hora en las últimas horas del día? Toma LME. Anoche también lo hizo durante la madrugada. No puedo dormirla entre toma y toma. '

CREACION LISTA CON TODAS LAS CATEGORIAS EN BBDD Neo4j

In [26]:
# Creación de la instancia de GraphPathFinder
finder = GraphPathFinder(url, username, password)
categorias = finder.get_all_theme_strings()
categorias = list(set(categorias))
print(categorias)
print(len(categorias))

['LACTANCIA POR ETAPAS', 'Punto Violeta', 'CONCEPTOS BÁSICOS', 'CÓMO PRODUCIR MÁS LECHE', 'SEGUIR PREPARÁNDOME', 'BULTOS EN EL PECHO', '¿TENGO POCA LECHE?', 'Sueño del bebé', 'PRUEBAS DIAGNÓSTICAS', 'MIS SENTIMIENTOS', 'SOBRE MI SALUD', 'PORTEO', 'EXTRACCIÓN DE LECHE', 'COSAS QUE HACE MI BEBÉ', 'Semana Mundial de la Lactancia', 'LACTANCIA MIXTA', 'PAÑALES SUCIOS', 'DOLOR EN EL PECHO', 'BACHES Y CRISIS', 'RELACTAR', '¿PUEDO...?', 'PECHO: FORMA Y TAMAÑO', 'DIFICULTADES DE AGARRE', 'ESTOY ENFERMA', 'LA DONACIÓN DE LECHE', 'ME HE QUEDADO EMBARAZADA', 'EN QUÉ DARLE LA LECHE', 'GUÍA DEL DOLOR AL AMAMANTAR', 'Emociones', 'Cómo gestionar mi mastitis', '¿QUÉ HAGO SI...?', 'LOS PRIMEROS DÍAS', 'INDUCCIÓN', 'ALIMENTOS SÓLIDOS', 'Guía de regalos', 'CONSERVACIÓN', 'A PARTIR DE 6 MESES', 'Preparar mi lactancia', 'GEMELOS, MELLIZOS', 'Coronavirus', 'EMBARAZO: SEGUIR LACTANDO', 'Planifica tu vuelta al trabajo', 'Lactancia artificial', 'DEJAR LA FÓRMULA', 'MI BEBÉ ES PREMATURO', 

IDENTIFICACION DE CLASIFICACION DEL MENSAJE DE LA USUARIA

entrada --> Mensaje de entrada

categorias --> Lista de todas las categorias identificadas en Neo4j

chainOpenAI --> Cadena de analisis con OpenAI

In [27]:
salidaOpen= clasificar_LLMs(entrada, categorias, chainOpenAI)

IDENTIFICACION NODO CON MAYOR SIMILITUD

In [28]:
# Creación de la instancia de GraphPathFinder
finder = GraphPathFinder(url, username, password)
ids = finder.get_node_ids_by_theme_string(salidaOpen)
ids = list(set(ids))
print(ids)
print(len(ids))

['t15']
1


Procesado para varios mensajes (Modo TEST)

In [31]:
for mensaje in mensajes[:1]:
    print("MENSAJE:", mensaje)
    print("-------------------------")
    # Query que deseas comparar con las propiedades de los nodos
    query = mensaje
    # Conexión a la base de datos Neo4j y cálculo de la similitud del coseno
    with GraphDatabase.driver(url, auth=(username, password)) as driver:
        embeddings = SentenceTransformer(model_name, **model_kwargs)

        with driver.session() as session:
            session.execute_write(calculate_similarity, query)

    print(f"Similitud del coseno calculada y asignada en la base de datos Neo4j para el query: {query}")

    # Identificación de los temas de la pregunta
    salidaOpen= clasificar_LLMs(mensaje, categorias, chainOpenAI)

    # Creación de la instancia de GraphPathFinder
    finder = GraphPathFinder(url, username, password)
    ids = finder.get_node_ids_by_theme_string(salidaOpen)

    # Lista de identificadores de nodo iniciales
    start_node_ids = ids  # Lista de nodos iniciales

    # Profundidad máxima de búsqueda
    Profund_Maxima_Nodos = 7

    # Número de caminos con mayor media de similaridad a mostrar
    n_top_paths = 5

    # Llamada a la función procesar_nodos
    resultados = procesar_nodos1(finder, start_node_ids, Profund_Maxima_Nodos, n_top_paths, mensaje, salidaOpen)

    mostrar_resultado(resultados[0], Profund_Maxima_Nodos)
    # Imprimir resultados
    #for resultado in resultados:
    #    print(resultado)

    # Cerrar la conexión
    finder.close()

MENSAJE: Y en caso que la semana que viene gane poco peso, (otros 40 gramos) seguiría siendo normal?

De hecho, hasta cuando sería normal?


-------------------------
Similitud del coseno calculada y asignada en la base de datos Neo4j para el query: Y en caso que la semana que viene gane poco peso, (otros 40 gramos) seguiría siendo normal?

De hecho, hasta cuando sería normal?


ID del Nodo Inicial: t14

Información del Camino (mostrando hasta 4 nodos):
  Nodo ID: t14, Similaridad: 0.8499402002327563
  Nodo ID: q751, Similaridad: 0.8549967665264577
  Nodo ID: c3211, Similaridad: 0.8327147471974291
  Nodo ID: r2315, Similaridad: 0.901515099613444

Media de Similaridad: 0.8597917033925218

Valores de string_es (mostrando hasta 4 valores):
  MI BEBÉ NO GANA PESO
  Una de la situaciones más agobiantes es que el bebé no gane peso, ¿en qué te puedo ayudar?
  ¿Mi leche no alimenta?
  
 La evidencia científica ha demostrado que no existen leches de mala calidad. Puedes recibir muchos come

RESUMEN RESULTADOS

In [32]:
# PREGUNTAS DE PRUEBA
print("Pregunta:", query, "\n")

Pregunta: Y en caso que la semana que viene gane poco peso, (otros 40 gramos) seguiría siendo normal?

De hecho, hasta cuando sería normal?

 



In [33]:
# Tematica identificada
print('Tematica:', salidaOpen)

Tematica: MI BEBÉ NO GANA PESO


In [34]:
# Nodo de tematica identificada
print('Nodo:', ids)

Nodo: ['t14']


In [35]:
# Nodos con mayor similaridad
for t in range(0, n_top_paths):
    tuplas = resultados[t]['path_info']
    # Extracción del primer elemento de cada tupla
    primeros_elementos= [tupla[0] for tupla in tuplas]
    # Mostrar la nueva lista
    print('Nodos camino identificado:', primeros_elementos)

Nodos camino identificado: ['t14', 'q751', 'c3211', 'r2315']
Nodos camino identificado: ['t14', 'q356', 'c2090', 'r1157']
Nodos camino identificado: ['t14', 'q751', 'c3208', 'r2312']
Nodos camino identificado: ['t14', 'q350', 'c2418', 'q180', 'c2408']
Nodos camino identificado: ['t14', 'q350', 'c2418', 'q180', 'c2417', 'r1446']


In [36]:
# CADENA TEXTOS
print('Numero de cadenas:', n_top_paths)
# Iterar sobre diferentes cadenas identificadas
for n in range(0, n_top_paths):
    print('Cadena: ',n, "Similaridad Media: ",resultados[n]['mean_similarity'], '\n')
    print("Pregunta: ", query)
    for t, texto in enumerate(resultados[n]['string_es_values']):
        tuplas = resultados[n]['path_info']
        # Extracción del primer elemento de cada tupla
        primeros_elementos= [tupla[0] for tupla in tuplas]
        print(t,': (Nodo:', primeros_elementos[t],')', texto)
    print('-------------------------')


Numero de cadenas: 5
Cadena:  0 Similaridad Media:  0.8597917033925218 

Pregunta:  Y en caso que la semana que viene gane poco peso, (otros 40 gramos) seguiría siendo normal?

De hecho, hasta cuando sería normal?


0 : (Nodo: t14 ) MI BEBÉ NO GANA PESO
1 : (Nodo: q751 ) Una de la situaciones más agobiantes es que el bebé no gane peso, ¿en qué te puedo ayudar?
2 : (Nodo: c3211 ) ¿Mi leche no alimenta?
3 : (Nodo: r2315 ) 
 La evidencia científica ha demostrado que no existen leches de mala calidad. Puedes recibir muchos comentarios de este tipo si tu bebé no engorda o es un bebé de medidas pequeñas, y sin duda, esto puede hacer que dudes de la calidad de tu leche o que a tu leche le falte algún elemento. 
 Puedes estar tranquila, tu leche es perfecta para tu bebé y si quieres, hay que buscar las causas por las que no esté ganando peso de manera óptima.
-------------------------
Cadena:  1 Similaridad Media:  0.8578926118955105 

Pregunta:  Y en caso que la semana que viene g

In [37]:
# Extracción del segundo elemento de cada tupla
segundos_elementos= [tupla[1] for tupla in tuplas]
print('Similaridad:', segundos_elementos) 

Similaridad: [0.8499402002327563, 0.8431711646431839, 0.8497322100265694, 0.8705979111346296, 0.8345551591594068, 0.8819772225814323]


In [38]:
# Similaridad media
print('Similaridad media:', resultados[0]['mean_similarity'])

Similaridad media: 0.8597917033925218


In [39]:
# Valores de string_es
print('Texto Nodos:', resultados[0]['string_es_values'])

Texto Nodos: ['MI BEBÉ NO GANA PESO', 'Una de la situaciones más agobiantes es que el bebé no gane peso, ¿en qué te puedo ayudar?', '¿Mi leche no alimenta?', '\n La evidencia científica ha demostrado que no existen leches de mala calidad. Puedes recibir muchos comentarios de este tipo si tu bebé no engorda o es un bebé de medidas pequeñas, y sin duda, esto puede hacer que dudes de la calidad de tu leche o que a tu leche le falte algún elemento. \n Puedes estar tranquila, tu leche es perfecta para tu bebé y si quieres, hay que buscar las causas por las que no esté ganando peso de manera óptima.']


SALIDAS EN FORMATO JSON

El objetivo es poder interactuar en la aplicacion WEB

In [40]:
import json

# CADENA TEXTOS
n_top_paths = 3  # Cambia esto según tu necesidad

# Crear una lista para almacenar los datos de salida
output_data = []

# Iterar sobre diferentes cadenas identificadas
for n in range(0, n_top_paths):
    cadena_data = {
        'Cadena': n,
        'Similaridad_Media': resultados[n]['mean_similarity'],
        'Pregunta': resultados[n]['query'],
        'Nodos_Textos': []
    }

    for t, texto in enumerate(resultados[n]['string_es_values']):
        tuplas = resultados[n]['path_info']
        primeros_elementos = [tupla[0] for tupla in tuplas]
        segundos_elementos = [tupla[1] for tupla in tuplas]
        nodo_texto = {
            'Nodo': primeros_elementos[t],
            'Similaridad': segundos_elementos[t],
            'Texto': texto
        }
        cadena_data['Nodos_Textos'].append(nodo_texto)

    output_data.append(cadena_data)

# Serializar los datos de salida como JSON
json_output = json.dumps(output_data, ensure_ascii=False, indent=4)

# Imprimir la salida JSON
print(json_output)

[
    {
        "Cadena": 0,
        "Similaridad_Media": 0.8597917033925218,
        "Pregunta": "Y en caso que la semana que viene gane poco peso, (otros 40 gramos) seguiría siendo normal?\n\nDe hecho, hasta cuando sería normal?\n\n",
        "Nodos_Textos": [
            {
                "Nodo": "t14",
                "Similaridad": 0.8499402002327563,
                "Texto": "MI BEBÉ NO GANA PESO"
            },
            {
                "Nodo": "q751",
                "Similaridad": 0.8549967665264577,
                "Texto": "Una de la situaciones más agobiantes es que el bebé no gane peso, ¿en qué te puedo ayudar?"
            },
            {
                "Nodo": "c3211",
                "Similaridad": 0.8327147471974291,
                "Texto": "¿Mi leche no alimenta?"
            },
            {
                "Nodo": "r2315",
                "Similaridad": 0.901515099613444,
                "Texto": "\n La evidencia científica ha demostrado que no existen l

In [41]:
# Cargar el JSON
data = json.loads(json_output)

# Imprimir el JSON formateado en pantalla
formatted_json = json.dumps(data, ensure_ascii=False, indent=4)
print(formatted_json)

[
    {
        "Cadena": 0,
        "Similaridad_Media": 0.8597917033925218,
        "Pregunta": "Y en caso que la semana que viene gane poco peso, (otros 40 gramos) seguiría siendo normal?\n\nDe hecho, hasta cuando sería normal?\n\n",
        "Nodos_Textos": [
            {
                "Nodo": "t14",
                "Similaridad": 0.8499402002327563,
                "Texto": "MI BEBÉ NO GANA PESO"
            },
            {
                "Nodo": "q751",
                "Similaridad": 0.8549967665264577,
                "Texto": "Una de la situaciones más agobiantes es que el bebé no gane peso, ¿en qué te puedo ayudar?"
            },
            {
                "Nodo": "c3211",
                "Similaridad": 0.8327147471974291,
                "Texto": "¿Mi leche no alimenta?"
            },
            {
                "Nodo": "r2315",
                "Similaridad": 0.901515099613444,
                "Texto": "\n La evidencia científica ha demostrado que no existen l

In [49]:
from sentence_transformers import SentenceTransformer
from neo4j import GraphDatabase

# Funcion para generar la salida resultados

def procesar_mensajes(mensajes, url, username, password, model_name, model_kwargs, categorias, chainOpenAI,
                      calculate_similarity, clasificar_LLMs, procesar_nodos1, mostrar_resultado):
    '''
    ENTRADA:
    mensajes --> Lista de mensajes
    url --> URL de la base de datos Neo4j
    username --> Nombre de usuario para la autenticación de la base de datos
    password --> Contraseña para la autenticación de la base de datos
    model_name --> Nombre del modelo para los embeddings
    model_kwargs --> Argumentos clave para el modelo (por ejemplo, dispositivo)
    categorias --> Lista de categorías para clasificar
    chainOpenAI --> Cadena de LangChain
    calculate_similarity --> Función para calcular la similaridad
    clasificar_LLMs --> Función para clasificar los mensajes
    procesar_nodos1 --> Función para procesar los nodos
    mostrar_resultado --> Función para mostrar los resultados

    SALIDA:
    resultados --> Lista de resultados
    '''
    for mensaje in mensajes:
        print("MENSAJE:", mensaje)
        print("-------------------------")

        # Query que deseas comparar con las propiedades de los nodos
        query = mensaje

        # Conexión a la base de datos Neo4j y cálculo de la similitud del coseno
        with GraphDatabase.driver(url, auth=(username, password)) as driver:
            embeddings = SentenceTransformer(model_name, **model_kwargs)

            with driver.session() as session:
                session.execute_write(calculate_similarity, query)

        print(f"Similitud del coseno calculada y asignada en la base de datos Neo4j para el query: {query}")

        # Identificación de los temas de la pregunta
        salidaOpen = clasificar_LLMs(mensaje, categorias, chainOpenAI)

        # Creación de la instancia de GraphPathFinder
        finder = GraphPathFinder(url, username, password)
        ids = finder.get_node_ids_by_theme_string(salidaOpen)

        # Lista de identificadores de nodo iniciales
        start_node_ids = ids

        # Profundidad máxima de búsqueda
        Profund_Maxima_Nodos = 7

        # Número de caminos con mayor media de similaridad a mostrar
        n_top_paths = 5

        # Llamada a la función procesar_nodos
        resultados = procesar_nodos1(finder, start_node_ids, Profund_Maxima_Nodos, n_top_paths, query, salidaOpen)

        if resultados:
            mostrar_resultado(resultados[0], Profund_Maxima_Nodos)
        else:
            print("No se encontraron resultados.")
        

        # Cerrar la conexión
        finder.close()

    return resultados

In [50]:
# Parámetros de conexión a la base de datos
#url = URL_NEO4J
#username = USERNAME_NEO4J
#password = PASSWORD_NEO4J

# Parámetros del modelo Sentence Transformer
#model_name = BAAI/bge-large-en
#model_kwargs = {'device': 'cpu'}

# Categorías y cadena OpenAI para clasificación
#categorias = categorias
#chainOpenAI = chainOpenAI

# Lista de mensajes para procesar
mensajes = ['Bebé de 8 meses, estoy tomando por indicación de la ginecóloga Natalben Lactancia desde el parto, pero son un pastón a final de mes! Debo seguir tomando las si sigo con lactancia? Hasta cuando? Gracias!', 'Perfecto, eso me parecía a mi también, de hecho con el primer hijo tomé únicamente yodocefol, pero tenía la duda al haberlo recomendado  la gine. Y debería tomarlo toda la lactancia? Hasta cuando se recomienda?', 'Hola buenas era porque tengo desde el lunes dolores menstruales sobretodo en la zona de los ovarios pero no me baja la regla aún sigo amamantando Ami pequeña el 17 hace los 6 meses ']

# Llamada a la función principal
resultados = procesar_mensajes(mensajes, url, username, password, model_name, model_kwargs, 
                               categorias, chainOpenAI, calculate_similarity, 
                               clasificar_LLMs, procesar_nodos1, mostrar_resultado)

# Imprimir los resultados
print(mensajes)
print(resultados)

MENSAJE: Bebé de 8 meses, estoy tomando por indicación de la ginecóloga Natalben Lactancia desde el parto, pero son un pastón a final de mes! Debo seguir tomando las si sigo con lactancia? Hasta cuando? Gracias!
-------------------------
Similitud del coseno calculada y asignada en la base de datos Neo4j para el query: Bebé de 8 meses, estoy tomando por indicación de la ginecóloga Natalben Lactancia desde el parto, pero son un pastón a final de mes! Debo seguir tomando las si sigo con lactancia? Hasta cuando? Gracias!
No se encontraron resultados.
MENSAJE: Perfecto, eso me parecía a mi también, de hecho con el primer hijo tomé únicamente yodocefol, pero tenía la duda al haberlo recomendado  la gine. Y debería tomarlo toda la lactancia? Hasta cuando se recomienda?
-------------------------
Similitud del coseno calculada y asignada en la base de datos Neo4j para el query: Perfecto, eso me parecía a mi también, de hecho con el primer hijo tomé únicamente yodocefol, pero tenía la duda al h

In [160]:
import json

def generar_salida_json(resultados, n_top_paths=3):
    """
    Genera una salida JSON a partir de una lista de resultados y un mensaje asociado.

    :param resultados: Lista de resultados a procesar.
    :param mensaje: Mensaje asociado a incluir en cada cadena de salida.
    :param n_top_paths: Número de elementos máximos a considerar en la salida.
    :return: Cadena en formato JSON con los datos procesados.
    """
    output_data = []

    # Asegurarse de que no se exceda el número de resultados disponibles
    n_top_paths = min(n_top_paths, len(resultados))

    for n in range(n_top_paths):
        cadena_data = {
            'Cadena': n,
            'Similaridad_Media': resultados[n]['mean_similarity'],
            'Pregunta': resultados[n]['query'],  # Incluir el mensaje aquí
            'Clasificacion': resultados[n]['clasificacion'],
            'Nodos_Textos': []
        }

        for t, texto in enumerate(resultados[n]['string_es_values']):
            tuplas = resultados[n]['path_info']
            primeros_elementos = [tupla[0] for tupla in tuplas]
            segundos_elementos = [tupla[1] for tupla in tuplas]

            nodo_texto = {
                'Nodo': primeros_elementos[t],
                'Similaridad': segundos_elementos[t], 
                'Texto': texto    
            }
            cadena_data['Nodos_Textos'].append(nodo_texto)

        output_data.append(cadena_data)

    # Serializar los datos de salida como JSON
    json_output = json.dumps(output_data, ensure_ascii=False, indent=4)
    return json_output

# Ejemplo de uso
# resultados = [...]  # Asegúrate de que esta variable contiene los resultados esperados
# mensaje = "Mensaje asociado"
# json_output = generar_salida_json(resultados, mensaje, n_top_paths=3)
# print(json_output)

#'start_node_id': start_node_id,
#'path_info': path_info,
#'mean_similarity': mean,
#'string_es_values': string_es_values,
#'max_similarity_node_info': max_similarity_node_info



In [96]:
json_output = generar_salida_json(resultados, n_top_paths=3)

# Cargar el JSON
data = json.loads(json_output)

# Imprimir el JSON formateado en pantalla
formatted_json = json.dumps(data, ensure_ascii=False, indent=4)
print(formatted_json)

[
    {
        "Cadena": 0,
        "Similaridad_Media": 0.863419727233861,
        "Pregunta": "Sí, parece que tu bebé tiene algo de retrognatia. Las posiciones verticales te pueden favorecer el agarre",
        "Nodos_Textos": [
            {
                "Nodo": "t16",
                "Similaridad": 0.8222326280815713,
                "Texto": "DIFICULTADES DE AGARRE"
            },
            {
                "Nodo": "q380",
                "Similaridad": 0.8529781263370453,
                "Texto": "¿Tu pecho está duro o sientes que está lleno de leche?"
            },
            {
                "Nodo": "c2162",
                "Similaridad": 0.8792558013469038,
                "Texto": "No, el pecho no está demasiado duro"
            },
            {
                "Nodo": "q381",
                "Similaridad": 0.871151566831059,
                "Texto": "¿Agarras a tu bebé por la cabeza para darle el pecho?"
            },
            {
                "Nodo": "c2

In [92]:
# Imprimir la salida JSON
print(json_output)

[
    {
        "Cadena": 0,
        "Similaridad_Media": 0.863419727233861,
        "Pregunta": "Sí, parece que tu bebé tiene algo de retrognatia. Las posiciones verticales te pueden favorecer el agarre",
        "Nodos_Textos": [
            {
                "Nodo": "t16",
                "Texto": "DIFICULTADES DE AGARRE"
            },
            {
                "Nodo": "q380",
                "Texto": "¿Tu pecho está duro o sientes que está lleno de leche?"
            },
            {
                "Nodo": "c2162",
                "Texto": "No, el pecho no está demasiado duro"
            },
            {
                "Nodo": "q381",
                "Texto": "¿Agarras a tu bebé por la cabeza para darle el pecho?"
            },
            {
                "Nodo": "c2164",
                "Texto": "Sí, lo acerco por la parte posterior de la cabeza"
            },
            {
                "Nodo": "r1237",
                "Texto": "\n Por todo lo que has indicado

Procesado masivo de mensajes de TEST

Lista con mensajes --> OBTENER ARCHIVO

In [97]:
def leer_csv_especifico(directorio, nombre_archivo):
    """
    Esta función lee un archivo .csv específico en el directorio dado.

    :param directorio: Ruta del directorio donde buscar el archivo
    :param nombre_archivo: Nombre del archivo .csv a leer
    :return: DataFrame de pandas con los datos del archivo .csv, o None si el archivo no se encuentra o hay un error
    """
    try:
        ruta_archivo_csv = os.path.join(directorio, nombre_archivo)
        
        # Verificar si el archivo existe
        if os.path.exists(ruta_archivo_csv) and ruta_archivo_csv.endswith('.csv'):
            df = pd.read_csv(ruta_archivo_csv)
            print(f"Archivo CSV leído: {nombre_archivo}")
            return df
        else:
            print(f"No se encontró el archivo '{nombre_archivo}' en el directorio.")
            return None
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
        return None

In [114]:
def leer_excel_especifico(directorio, nombre_archivo):
    """
    Esta función lee un archivo .xlsx específico en el directorio dado.

    :param directorio: Ruta del directorio donde buscar el archivo
    :param nombre_archivo: Nombre del archivo .xlsx a leer
    :return: DataFrame de pandas con los datos del archivo .xlsx, o None si el archivo no se encuentra o hay un error
    """
    try:
        ruta_archivo_excel = os.path.join(directorio, nombre_archivo)
        
        # Verificar si el archivo existe
        if os.path.exists(ruta_archivo_excel) and ruta_archivo_excel.endswith('.xlsx'):
            df = pd.read_excel(ruta_archivo_excel)
            print(f"Archivo Excel leído: {nombre_archivo}")
            return df
        else:
            print(f"No se encontró el archivo '{nombre_archivo}' en el directorio.")
            return None
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
        return None

In [108]:
def listar_achivos(directorio):
    """
    Esta función lista los archivos en el directorio dado.

    :param directorio: Ruta del directorio a listar
    """
    try:
        # Listar todos los archivos y directorios en el directorio dado
        archivos = os.listdir(directorio)
        print(f"Archivos en '{directorio}':")
        
        # Iterar a través de los archivos y directorios
        for archivo in archivos:
            # Ruta completa del archivo
            ruta_completa = os.path.join(directorio, archivo)
            
            # Verificar si es un archivo o un directorio
            if os.path.isfile(ruta_completa):
                print(f"Archivo: {archivo}")
            else:
                print(f"Directorio: {archivo}")
    except Exception as e:
        print(f"Error: {e}")

In [109]:
# Uso del script
directorio = '/media/ubuntu/Linux3/02_TFM/DATOS_APP'
listar_archivos(directorio)

Archivos en '/media/ubuntu/Linux3/02_TFM/DATOS_APP':
Archivo: conversations_no_chitchat.csv
Archivo: conversations.csv


In [100]:
# Uso del script
df = leer_csv_especifico(directorio, 'conversations.csv')

Archivo CSV leído: conversations.csv


In [103]:
df.head(1)

,treeName,msg_id,db_user_id,link,message,user_id,expert,tree,time,discard,classified,expert_old,empty,conversation_id
0,Classifying,5e5ea4db4196ca0011c5fdbf,33020,Classifying,Hola! Buenas tardes! Una pregunta: Una mami de...,000121b28e5a9eda9f9fecf5,user,Classifying,1.583261e+09,Classifying,True,1MP_Laura,0,1


In [115]:
# Uso del script
directorio = '/media/ubuntu/Linux3/02_TFM/TEST_BIBLIOTECA/MODELOS'
listar_archivos(directorio)

Archivos en '/media/ubuntu/Linux3/02_TFM/TEST_BIBLIOTECA/MODELOS':
Archivo: list.gbnf
Archivo: AAU_Neo4j_01.ipynb
Archivo: SALIDA.xlsx
Archivo: .env
Archivo: CLA03.ipynb
Directorio: __pycache__
Archivo: CLA04_TEST.ipynb
Archivo: AAU_Neo4j_02.ipynb
Archivo: llama-2-13b-chat.Q3_K_M.gguf
Archivo: MODELO_Mistral7b.ipynb
Archivo: MODELO_OpenAI.ipynb
Archivo: openai_utils.py
Archivo: CLA02.ipynb
Archivo: CLA04.ipynb
Archivo: AAU_langchain_neo4jvector.ipynb
Archivo: MODELO_Llama2.ipynb
Archivo: CLASIFICACION.ipynb
Archivo: Angel.pdf
Archivo: estructura1.json
Archivo: CLA01_OpenAI.ipynb
Archivo: a.xlsx


In [116]:
# Uso del script
df = leer_excel_especifico(directorio, 'a.xlsx')
df.head(1)

Archivo Excel leído: a.xlsx


,Unnamed: 0,msg_id,user_id,time,message,expert,treeName
0,5,64f59763872fff8b680b361d,0002d0367e95ae169d1ee7fa,1.693817e+09,"Hola buenos días , me siento fatal hoy empiezo...",user,LA VUELTA AL TRABAJO


In [119]:
# Selecciona solo tres columnas
nuevo_df = df[['message', 'treeName','expert']]
nuevo_df.head(1)

,message,treeName,expert
0,"Hola buenos días , me siento fatal hoy empiezo...",LA VUELTA AL TRABAJO,user


In [120]:
# Obtener los valores únicos de la columna
valores_unicostreeName = df['treeName'].unique()
valores_unicosexpert = df['expert'].unique()
# Convertir a lista si es necesario
valores_unicostreeNameL = list(valores_unicostreeName)
valores_unicosexpertL = list(valores_unicosexpert)
# Mostrar la lista
print(valores_unicostreeNameL)
print(valores_unicosexpertL)

[' LA VUELTA AL TRABAJO', 'DOLOR EN EL PECHO', 'MI BEBÉ NO GANA PESO', 'PAÑALES SUCIOS', '¿PUEDO...?', 'Menstruación y fertilidad', 'DEJAR DE DAR EL PECHO', 'LACTANCIA POR ETAPAS', 'BACHES Y CRISIS', 'LOS PRIMEROS DÍAS', 'SUEÑO Y LACTANCIA', 'CONSERVACIÓN ', 'Durante mi embarazo', 'RELACTAR ', 'LACTANCIA MIXTA', 'DIFICULTADES DE AGARRE', ' BULTOS EN EL PECHO', 'CONCEPTOS BÁSICOS', 'ALIMENTOS SÓLIDOS', 'PRODUCTOS PARA LA LACTANCIA', 'COSAS QUE HACE MI BEBÉ', '¿QUÉ HAGO SI...?', 'SOBRE MI SALUD', 'PRUEBAS DIAGNÓSTICAS ', 'SALUD DE MI BEBÉ', 'CÓMO PRODUCIR MÁS LECHE', 'Coronavirus', 'EL DESTETE POR ETAPAS', 'LA DONACIÓN DE LECHE', 'EMBARAZO: SEGUIR LACTANDO ', 'MI BEBÉ MAMA MUCHO', 'SKFin', 'ME HE QUEDADO EMBARAZADA', 'EXTRACCIÓN DE LECHE', 'PECHO: FORMA Y TAMAÑO', 'MIS SENTIMIENTOS ', 'A PARTIR DE 6 MESES ', 'GRIETAS Y HERIDAS', '¿TENGO POCA LECHE?', 'EN QUÉ DARLE LA LECHE', 'ESTOY ENFERMA ', 'MI BEBÉ ES PREMATURO', 'INDUCCIÓN', 'No Clasificable', 'GUÍA DEL DOLOR AL AMAMANTAR', 'PORTEO',

In [125]:
# Comparacion categorias
lista1 = valores_unicostreeName
lista2 = categorias

# Encuentra elementos únicos en lista1
unicos_en_lista1 = [item for item in lista1 if item not in lista2]

# Encuentra elementos únicos en lista2
unicos_en_lista2 = [item for item in lista2 if item not in lista1]

# Encuentra elementos comunes en ambas listas
elementos_comunes = [item for item in lista1 if item in lista2]

# Imprimir resultados
print("Longitud:", len(unicos_en_lista1),"Elementos únicos en Archivo Mensajes:", unicos_en_lista1)
print("Longitud:", len(unicos_en_lista2), "Elementos únicos en Neo4j:", unicos_en_lista2)
print("Longitud:", len(elementos_comunes),"Elementos comunes en ambas listas:", elementos_comunes)


Longitud: 35 Elementos únicos en Archivo Mensajes: [' LA VUELTA AL TRABAJO', 'MI BEBÉ NO GANA PESO', 'PAÑALES SUCIOS', 'Menstruación y fertilidad', 'LOS PRIMEROS DÍAS', 'SUEÑO Y LACTANCIA', 'CONSERVACIÓN ', 'RELACTAR ', ' BULTOS EN EL PECHO', 'CONCEPTOS BÁSICOS', 'ALIMENTOS SÓLIDOS', 'COSAS QUE HACE MI BEBÉ', '¿QUÉ HAGO SI...?', 'PRUEBAS DIAGNÓSTICAS ', 'SALUD DE MI BEBÉ', 'CÓMO PRODUCIR MÁS LECHE', 'LA DONACIÓN DE LECHE', 'EMBARAZO: SEGUIR LACTANDO ', 'MI BEBÉ MAMA MUCHO', 'SKFin', 'EXTRACCIÓN DE LECHE', 'PECHO: FORMA Y TAMAÑO', 'MIS SENTIMIENTOS ', 'A PARTIR DE 6 MESES ', 'EN QUÉ DARLE LA LECHE', 'ESTOY ENFERMA ', 'MI BEBÉ ES PREMATURO', 'INDUCCIÓN', 'No Clasificable', 'GUÍA DEL DOLOR AL AMAMANTAR', 'Preparar mi lactancia ', 'Sueño del bebé', 'ÁRBOL EMOCIONES', 'DEJAR LA FÓRMULA', 'SEGUIR PREPARÁNDOME']
Longitud: 39 Elementos únicos en Neo4j: ['EMBARAZO: SEGUIR LACTANDO', 'CONSERVACIÓN', 'Semana Mundial de la Lactancia', 'MIS SENTIMIENTOS', 'LOS PRIMEROS DÍAS', 'MI BEBÉ MAMA MUCHO

In [126]:
# Añadir una categoria a la lista
elementos_comunes.append('No Clasificable')

In [131]:
# Crear un nuevo DataFrame con solo los valores que estan en la lista de elementos comunes para poder analizarlos
nuevo_df1 = nuevo_df[nuevo_df['treeName'].isin(elementos_comunes)]
nuevo_df1.shape

(131383, 3)

Analisis de mensajes 

Mensaje --> Clasificacion donde envia sistema | Nodo donde envia | Camino identificado | Clasificación Experta

In [250]:
# Extraer los mensajes de la columna message
mensajes = nuevo_df1['message'].to_list()[0:3]
print(mensajes)

['Bebé de 8 meses, estoy tomando por indicación de la ginecóloga Natalben Lactancia desde el parto, pero son un pastón a final de mes! Debo seguir tomando las si sigo con lactancia? Hasta cuando? Gracias!', 'Perfecto, eso me parecía a mi también, de hecho con el primer hijo tomé únicamente yodocefol, pero tenía la duda al haberlo recomendado  la gine. Y debería tomarlo toda la lactancia? Hasta cuando se recomienda?', 'Hola buenas era porque tengo desde el lunes dolores menstruales sobretodo en la zona de los ovarios pero no me baja la regla aún sigo amamantando Ami pequeña el 17 hace los 6 meses ']


In [251]:
# Llamada a la función principal
resultados = procesar_mensajes(mensajes, url, username, password, model_name, model_kwargs, 
                               categorias, chainOpenAI, calculate_similarity, 
                               clasificar_LLMs, procesar_nodos1, mostrar_resultado)

PROCESANDO MENSAJE 1: Bebé de 8 meses, estoy tomando por indicación de la ginecóloga Natalben Lactancia desde el parto, pero son un pastón a final de mes! Debo seguir tomando las si sigo con lactancia? Hasta cuando? Gracias!
-------------------------
Similitud del coseno calculada para el query: Bebé de 8 meses, estoy tomando por indicación de la ginecóloga Natalben Lactancia desde el parto, pero son un pastón a final de mes! Debo seguir tomando las si sigo con lactancia? Hasta cuando? Gracias!
Error al procesar el mensaje 1: Bebé de 8 meses, estoy tomando por indicación de la ginecóloga Natalben Lactancia desde el parto, pero son un pastón a final de mes! Debo seguir tomando las si sigo con lactancia? Hasta cuando? Gracias!
Detalle del error: list index out of range
PROCESANDO MENSAJE 2: Perfecto, eso me parecía a mi también, de hecho con el primer hijo tomé únicamente yodocefol, pero tenía la duda al haberlo recomendado  la gine. Y debería tomarlo toda la lactancia? Hasta cuando se r

In [252]:
resultados


[]

In [198]:
json_output = generar_salida_json(resultados, n_top_paths=3)

In [1]:
# Cargar el JSON
data = json.loads(json_output)

# Imprimir el JSON formateado en pantalla
formatted_json = json.dumps(data, ensure_ascii=False, indent=4)
print(formatted_json)

NameError: name 'json' is not defined

In [186]:
# CORRECTA

def imprimir_resultados_OK(indice_mensaje, mensaje, resultados, profundidad_maxima):
    print(f"RESULTADOS PARA EL MENSAJE {indice_mensaje} ('{mensaje}'):")
    print("Profundidad Máxima:", profundidad_maxima)
    print("--------------------------------------------------")
    for idx, resultado in enumerate(resultados, 1):
        print(f"Resultado {idx}: {resultado}")
    print("--------------------------------------------------\n")

def procesar_mensajes_OK(mensajes, url, username, password, model_name, model_kwargs, categorias, chainOpenAI,
                      calculate_similarity, clasificar_LLMs, procesar_nodos1, mostrar_resultado):
    resultados_globales = []
    for indice, mensaje in enumerate(mensajes, 1):
        print(f"PROCESANDO MENSAJE {indice}: {mensaje}")
        print("-------------------------")

        query = mensaje

        try:
            with GraphDatabase.driver(url, auth=(username, password)) as driver:
                embeddings = SentenceTransformer(model_name, **model_kwargs)

                with driver.session() as session:
                    session.execute_write(calculate_similarity, query)

            print(f"Similitud del coseno calculada para el query: {query}")

            salidaOpen = clasificar_LLMs(mensaje, categorias, chainOpenAI)

            finder = GraphPathFinder(url, username, password)
            ids = finder.get_node_ids_by_theme_string(salidaOpen)

            start_node_ids = ids
            Profund_Maxima_Nodos = 7
            n_top_paths = 5

            resultados = procesar_nodos1(finder, start_node_ids, Profund_Maxima_Nodos, n_top_paths, query, salidaOpen)
            mostrar_resultado(resultados[0], Profund_Maxima_Nodos)

            imprimir_resultados_OK(indice, mensaje, resultados, Profund_Maxima_Nodos)

            resultados_globales.append((indice, mensaje, resultados))

            finder.close()

        except Exception as e:
            print(f"Error al procesar el mensaje {indice}: {mensaje}")
            print(f"Detalle del error: {e}")

    return resultados_globales

In [194]:
# CORRECTA

def generar_salida_json_OK(resultados, n_top_paths=3):
    """
    Genera una salida JSON a partir de una lista de resultados y un mensaje asociado.

    :param resultados: Lista de resultados a procesar. Cada elemento es una tupla (indice, mensaje, resultados).
    :param n_top_paths: Número de elementos máximos a considerar en la salida.
    :return: Cadena en formato JSON con los datos procesados.
    """
    output_data = {
        'Mensajes_Procesados': []
    }

    for indice, mensaje, datos_resultado in resultados:
        # Asegurarse de que no se exceda el número de resultados disponibles
        n_top_paths = min(n_top_paths, len(datos_resultado))

        mensaje_data = {
            'Indice': indice,
            'Contenido': mensaje,
            'Clasificacion': datos_resultado[0]['clasificacion'] if datos_resultado else None,
            'Cadenas_Analizadas': []
        }

        for n in range(n_top_paths):
            cadena_data = {
                'Indice_Cadena': n,
                'Similaridad_Media': datos_resultado[n]['mean_similarity'],
                'Nodos': []
            }

            for t, texto in enumerate(datos_resultado[n]['string_es_values']):
                tuplas = datos_resultado[n]['path_info']
                primeros_elementos = [tupla[0] for tupla in tuplas]
                segundos_elementos = [tupla[1] for tupla in tuplas]

                nodo_data = {
                    'Indice_Nodo': primeros_elementos[t],
                    'Similaridad': segundos_elementos[t],
                    'Texto': texto
                }
                cadena_data['Nodos'].append(nodo_data)

            mensaje_data['Cadenas_Analizadas'].append(cadena_data)

        output_data['Mensajes_Procesados'].append(mensaje_data)

    # Serializar los datos de salida como JSON
    json_output = json.dumps(output_data, ensure_ascii=False, indent=4)
    return json_output


In [238]:
# Define a prompt template for generating prompts.
# Define una plantilla de prompt para generar preguntas.
prompt_template1 = """Como experto en lactancia, tu tarea es identificar y formular preguntas clave que aún no han sido mencionadas por el usuario,
pero que son esenciales para comprender y responder completamente a su consulta sobre '{theme}'.
\nPregunta del usuario: '{user_message}'\n
Temática: Lactancia y '{theme}'.
\nFormato de salida deseado: Preguntas adicionales para el usuario
- ¿Pregunta 1 relacionada con el concepto no mencionado?
- ¿Pregunta 2 relacionada con otro concepto no mencionado?
...
Elabora preguntas que ayuden a profundizar en los aspectos no cubiertos por la pregunta original del usuario, basándote en tu conocimiento especializado en lactancia.
"""

prompt_template2 = """Como experto en lactancia, tu tarea es analizar y extraer la información relevante de la pregunta del usuario,
relacionada con el tema '{theme}', que puede ser crucial para resolver su consulta.
\nPregunta del usuario: '{user_message}'\n
Temática: Lactancia y '{theme}'.
\nFormato de salida deseado: Lista de información relevante contenida en la pregunta del usuario
- [Información relevante 1]
- [Información relevante 2]
...
Identifica y enumera los elementos de información en la pregunta del usuario que son esenciales para entender y responder adecuadamente a su consulta, basándote en tu conocimiento especializado en lactancia.
"""

# Specify the input variables used in the template.
# Especifica las variables de entrada utilizadas en la plantilla.
input_variables = ['theme', 'user_message']

# Create a PromptTemplate object with the specified template and input variables.
# Crea un objeto PromptTemplate con la plantilla y las variables de entrada especificadas.
prompt1 = PromptTemplate(template=prompt_template1, input_variables=input_variables)
prompt2 = PromptTemplate(template=prompt_template2, input_variables=input_variables)

In [239]:
prompt

PromptTemplate(input_variables=['theme', 'user_message'], template="Como experto en lactancia, tu tarea es identificar y formular preguntas clave que aún no han sido mencionadas por el usuario,\npero que son esenciales para comprender y responder completamente a su consulta sobre '{theme}'.\n\nPregunta del usuario: '{user_message}'\n\nTemática: Lactancia y '{theme}'.\n\nFormato de salida deseado: Preguntas adicionales para el usuario\n- ¿Pregunta 1 relacionada con el concepto no mencionado?\n- ¿Pregunta 2 relacionada con otro concepto no mencionado?\n...\nElabora preguntas que ayuden a profundizar en los aspectos no cubiertos por la pregunta original del usuario, basándote en tu conocimiento especializado en lactancia.\n")

In [249]:
resultados[0][1]

'Sí, parece que tu bebé tiene algo de retrognatia. Las posiciones verticales te pueden favorecer el agarre'

In [240]:
user_message = resultados[0][1]

In [241]:
resultados[0][2][0]['string_es_values'][0]

'DIFICULTADES DE AGARRE'

In [242]:
theme = resultados[0][2][0]['string_es_values'][0]

In [243]:
chainOpenAI1 = prompt1 | modelOpenAI | StrOutputParser()
chainOpenAI2 = prompt2 | modelOpenAI | StrOutputParser()

In [244]:
def clasificar_LLMs(Tema, Mensaje, chain):
    '''Función de clasificación
    Entrada:
        texto -> Conversación Usuaria
        categorias --> Lista de categorias donde se ha de clasificar
        chain --> Cadena de LangChain
    respuesta:
        Respuesta del modelo
    '''
    response = chain.invoke({'theme':Tema, 'user_message':Mensaje})
    return response

In [245]:
print(clasificar_LLMs(theme, user_message, chainOpenAI1))

1. ¿Has notado alguna otra dificultad de agarre además de la retrognatia?
2. ¿Cuánto tiempo llevas experimentando estas dificultades de agarre?
3. ¿Has intentado diferentes posiciones de lactancia para mejorar el agarre?
4. ¿Has recibido algún consejo o ayuda profesional para abordar estas dificultades de agarre?
5. ¿Has notado algún cambio en la alimentación o el crecimiento de tu bebé debido a estas dificultades de agarre?
6. ¿Has considerado el uso de dispositivos de ayuda, como pezoneras o protectores de pezones, para mejorar el agarre?
7. ¿Has experimentado dolor o molestias durante la lactancia debido a estas dificultades de agarre?
8. ¿Has notado alguna relación entre las dificultades de agarre y la producción de leche materna?
9. ¿Has intentado técnicas de estimulación o masaje para ayudar a tu bebé a abrir más la boca durante el agarre?
10. ¿Has consultado con otros expertos en lactancia o grupos de apoyo para obtener más orientación sobre estas dificultades de agarre?


In [246]:
print(clasificar_LLMs(theme, user_message, chainOpenAI2))

- El bebé tiene dificultades de agarre.
- El bebé tiene retrognatia.
- Las posiciones verticales pueden favorecer el agarre.


In [221]:
user_message

['Sí, parece que tu bebé tiene algo de retrognatia. Las posiciones verticales te pueden favorecer el agarre']